In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait as wait

In [2]:
import time
import pandas as pd

In [3]:
current_version = "99"
chrome_options = Options()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--start-maximized')
# chromedriver_path = f"\\\\eip.esunbank.com.tw@SSL\DavWWWRoot\sites\C010\DocLib1\kentsai\chromedriver\{current_version}\chromedriver.exe"
chromedriver_path = r"C:\Users\ESB21313\[RPA]\chromedriver.exe"

In [4]:
driver = webdriver.Chrome(chromedriver_path, options = chrome_options)
driver.get("https://hrisweb.esunbank.com.tw:8443/psp/HRISP/EMPLOYEE/HRMS/?cmd=logout")

<ipython-input-4-0d1ecfb8cc5e>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path, options = chrome_options)


In [5]:
# 請自行輸入帳密
# 至以下畫面

![title](./員工自助服務.jpg)

In [7]:
# 到員工自助畫面 再執行以下

#點選差勤管理
driver.find_element(By.XPATH, "//*[@id='win0divPTNUI_LAND_REC_GROUPLET$1']").click()

In [24]:
#點選 差勤資料
time.sleep(0.5)
driver.find_element(By.XPATH, "//*[@id='win4divPTGP_STEP_DVW_PTGP_STEP_BTN_GB$4']").click()

In [25]:
#點選 檢視出勤紀錄
time.sleep(0.5)
driver.find_element(By.XPATH, "//*[@id='win4divPTGP_STEP_DVW_PTGP_STEP_BTN_GB$1$$6']").click()

# 如果每個月點的話，應該會顯示當月的出勤紀錄，如下畫面

![title](./出勤紀錄.jpg)

In [26]:
from selenium.common.exceptions import NoSuchElementException

In [27]:
#先進入iframe = =
driver.switch_to.frame("main_target_win0")

In [28]:
#找出時間範圍內所有報公務加班的日期
ot_date = []

#抓取紅色公務的字 ，即公務未報加班
while True:
    #選擇該table之元素
    ot_table_rows_ele = driver.find_elements(By.XPATH, "//*[@id='tdgbrAB_ATT_DTL$0']/tbody/tr")
    
    for i in ot_table_rows_ele:
        cur_row_off_field = i.find_element(By.XPATH, ".//td[10]/div/span")
        if cur_row_off_field.value_of_css_property('color') =='rgba(237, 28, 36, 1)':
            ot_date.append(i.find_element(By.XPATH, ".//td[1]/div/span").text)
            
    # 按下一頁按鈕，直到不能按為止
    try:
        driver.find_element(By.XPATH, "//*[@id='AB_ATT_DTL$hdown$0']").click()
        time.sleep(1)
    except NoSuchElementException:
        #如果不能按下一頁(即最後一頁)則停止蒐資料
        break        

In [29]:
ot_date

[]

In [15]:
from selenium.webdriver.support.ui import WebDriverWait

In [16]:
# start_time_field = driver.execute_script("return document.getElementById('Z_GP_OT_DERIV_START_TIME')")

In [17]:
# driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_START_TIME']").click()

In [18]:
for i in ot_date:
    # iframe跳脫
    driver.switch_to.default_content()

    #點選加班申請
    driver.find_element(By.XPATH, "//*[@id='win4divPTGP_STEP_DVW_PTGP_STEP_BTN_GB$2']").click()
    time.sleep(1)

    #開始日期
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_BGN_DT']").clear()
    time.sleep(0.1)
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_BGN_DT']").send_keys(i)
    #開始時間
    #點這邊會載入一些東西 要等一下
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_START_TIME']").click()
    driver.implicitly_wait(2)
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_START_TIME']").send_keys('18:30')
    time.sleep(0.1)
    #結束日期
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_END_DT']").clear()
    time.sleep(0.1)
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_END_DT']").send_keys(i)
    time.sleep(0.1)
    # 取得下班時間
    off_work_time = driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_AB_PUNCH_DTTM_OUT']").text
    off_time = off_work_time.split()[1]
    time.sleep(0.1)
    # 結束時間
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_END_TIME']").send_keys(off_time)
    time.sleep(0.1)
    # 加班事由
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_COMMENTS']").send_keys("RPA開發")
    time.sleep(0.1)
    # 點提交
    driver.find_element(By.XPATH, "//*[@id='Z_GP_OT_DERIV_SUBMIT_PB']").click()
    time.sleep(0.1)

    #先進入iframe...
    iframe = driver.find_element(By.XPATH, "/html/body/form/div[1]/div[3]/div[2]/div/div/iframe")
    driver.switch_to.frame(iframe)
    
    # 確定提交
    button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='DERIVED_ABS_SS_YES']"))
    )
    button.click()
    
    time.sleep(1)

overtime_div = driver.find_element(By.XPATH, "//*[@id='l0AB_ATT_DTL$0']")

html = overtime_div.get_attribute("outerHTML")

html

overtime_dfs = pd.read_html(html)

overtime_dfs[0]